<a href="https://colab.research.google.com/github/Zahra-Mazinani/Data_Divar/blob/main/DivarFeatureDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In the name of Allah


In [ ]:
!pip install pyspark
!pip install parsivar
!pip install catboost shap 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 64 kB/s 
     |████████████████████████████████| 199 kB 83.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=af7211b38a600b4671a0bfd66a5fac224e56dc2edc5e2902e17db4353b6a8dcf
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 36.2 MB 1.2 MB/s 
     |████████████████████████████████| 1.5 MB 62.2 MB/s 
  Created wheel for parsivar: filename=parsivar-0.2.3-py3-none-any.whl size=36492972 sha256=21673b0d01db5fccb463647b6411c86b1f269fea63c829d9f4fb4bc699f25496
  Stored in directory: /root/.cache/pip/wheels/ae/67/7a/49cbf08f64d3f76a26eceaf0e481a40e233f05d4356875cbed
  Created wh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip "/content/drive/MyDrive/Colab Notebooks/Divar/DMC-phase1-DataSets-20211225T055615Z-001.zip" 

Mounted at /content/drive
Archive:  /content/drive/MyDrive/Colab Notebooks/Divar/DMC-phase1-DataSets-20211225T055615Z-001.zip
  inflating: DMC-phase1-DataSets/reject_reasons_info.csv  
  inflating: DMC-phase1-DataSets/DMC-Phase1-Validation.parquet  
  inflating: DMC-phase1-DataSets/DMC-Train.parquet  


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import *

import pandas as pd
import numpy as np

In [ ]:
path = "/content/DMC-phase1-DataSets"

spark = SparkSession.builder.appName('Divar').getOrCreate()

df_train = spark.read.parquet(path + '/DMC-Train.parquet')
df_test = spark.read.parquet(path + '/DMC-Phase1-Validation.parquet')

In [ ]:
schema = StructType([ 
    StructField("body_status",StringType(),True), 
    StructField("brand",StringType(),True), 
    StructField("brand_model",StringType(),True), 
    StructField("category",StringType(),True), 
    StructField("color",StringType(),True), 
    StructField("description",StringType(),True), 
    StructField("document",StringType(),True), 
    StructField("gearbox",StringType(),True), 
    StructField("new_price",IntegerType(),True), 
    StructField("post_type",StringType(),True), 
    StructField("selling_type",StringType(),True), 
    StructField("third_party_insurance_deadline",IntegerType(),True), 
    StructField("tit le",StringType(),True), 
    StructField("usage", IntegerType(), True),
    StructField("year", StringType(), True)
  ])

df_train = df_train.withColumn("post_data", from_json("post_data", schema))\
    .select(col('post_id'),
            col('review_label'),
            col('post_data.*'))
df_test = df_test.withColumn("post_data", from_json("post_data", schema))\
    .select(col('post_id'),
            col('post_data.*'))

df_train = df_train.withColumn("review_label", when(df_train.review_label == "accept","1")\
      .when(df_train.review_label == "reject","0"))
df_train = df_train.withColumn("review_label",col("review_label").cast(IntegerType()))

## Preproccecing

### Utils

In [ ]:
from __future__ import unicode_literals

import collections
import gc      #garbage
import json
import re
import os

import numpy as np
import pandas as pd
import parsivar
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [ ]:
def read_json(path, n_lines_to_read=None):
    """
    read json file line by line iteratively (generator function).
    use this function to read json files when you have memory limitations.
    """

    with open(path) as f:
        for i, line in enumerate(tqdm(f)):
            if n_lines_to_read == i:
                break
            yield json.loads(line)

In [ ]:
parsivar_normalizer = parsivar.Normalizer(statistical_space_correction=True)

char_mappings = {
    "٥": "5",
    "А": "a",
    "В": "b",
    "Е": "e",
    "Н": "h",
    "Р": "P",
    "С": "C",
    "Т": "T",
    "а": "a",
    "г": "r",
    "е": "e",
    "к": "k",
    "м": "m",
    "о": "o",
    "р": "p",
    "ڈ": "د",
    "ڇ": "چ",
    # Persian numbers (will be raplaced by english one)
    "۰": "0",
    "۱": "1",
    "۲": "2",
    "۳": "3",
    "۴": "4",
    "۵": "5",
    "۶": "6",
    "۷": "7",
    "۸": "8",
    "۹": "9",
    ".": ".",
    # Arabic numbers (will be raplaced by english one)
    "٠": "0",
    "١": "1",
    "٢": "2",
    "٣": "3",
    "٤": "4",
    "٥": "5",
    "٦": "6",
    "٧": "7",
    "٨": "8",
    "٩": "9",
    # Special Arabic Characters (will be replaced by persian one)
    "ك": "ک",
    "ى": "ی",
    "ي": "ی",
    "ؤ": "و",
    "ئ": "ی",
    "إ": "ا",
    "أ": "ا",
    "آ": "ا",
    "ة": "ه",
    "ء": "ی",
    # French alphabet (will be raplaced by english one)
    "à": "a",
    "ä": "a",
    "ç": "c",
    "é": "e",
    "è": "e",
    "ê": "e",
    "ë": "e",
    "î": "i",
    "ï": "i",
    "ô": "o",
    "ù": "u",
    "û": "u",
    "ü": "u",
    # Camma (will be replace by dots for floating point numbers)
    ",": ".",
    # And (will be replace by dots for floating point numbers)
    "&": " and ",
    # Vowels (will be removed)
    "ّ": "",  # tashdid
    "َ": "",  # a
    "ِ": "",  # e
    "ُ": "",  # o
    "ـ": "",  # tatvil
    # Spaces
    "‍": "",  # 0x9E -> ZERO WIDTH JOINER
    "‌": " ",  # 0x9D -> ZERO WIDTH NON-JOINER
    # Arabic Presentation Forms-A (will be replaced by persian one)
    "ﭐ": "ا",
    "ﭑ": "ا",
    "ﭖ": "پ",
    "ﭗ": "پ",
    "ﭘ": "پ",
    "ﭙ": "پ",
    "ﭞ": "ت",
    "ﭟ": "ت",
    "ﭠ": "ت",
    "ﭡ": "ت",
    "ﭺ": "چ",
    "ﭻ": "چ",
    "ﭼ": "چ",
    "ﭽ": "چ",
    "ﮊ": "ژ",
    "ﮋ": "ژ",
    "ﮎ": "ک",
    "ﮏ": "ک",
    "ﮐ": "ک",
    "ﮑ": "ک",
    "ﮒ": "گ",
    "ﮓ": "گ",
    "ﮔ": "گ",
    "ﮕ": "گ",
    "ﮤ": "ه",
    "ﮥ": "ه",
    "ﮦ": "ه",
    "ﮪ": "ه",
    "ﮫ": "ه",
    "ﮬ": "ه",
    "ﮭ": "ه",
    "ﮮ": "ی",
    "ﮯ": "ی",
    "ﮰ": "ی",
    "ﮱ": "ی",
    "ﯼ": "ی",
    "ﯽ": "ی",
    "ﯾ": "ی",
    "ﯿ": "ی",
    # Arabic Presentation Forms-B (will be removed)
    "ﹰ": "",
    "ﹱ": "",
    "ﹲ": "",
    "ﹳ": "",
    "ﹴ": "",
    "﹵": "",
    "ﹶ": "",
    "ﹷ": "",
    "ﹸ": "",
    "ﹹ": "",
    "ﹺ": "",
    "ﹻ": "",
    "ﹼ": "",
    "ﹽ": "",
    "ﹾ": "",
    "ﹿ": "",
    # Arabic Presentation Forms-B (will be replaced by persian one)
    "ﺀ": "ی",
    "ﺁ": "ا",
    "ﺂ": "ا",
    "ﺃ": "ا",
    "ﺄ": "ا",
    "ﺅ": "و",
    "ﺆ": "و",
    "ﺇ": "ا",
    "ﺈ": "ا",
    "ﺉ": "ی",
    "ﺊ": "ی",
    "ﺋ": "ی",
    "ﺌ": "ی",
    "ﺍ": "ا",
    "ﺎ": "ا",
    "ﺏ": "ب",
    "ﺐ": "ب",
    "ﺑ": "ب",
    "ﺒ": "ب",
    "ﺓ": "ه",
    "ﺔ": "ه",
    "ﺕ": "ت",
    "ﺖ": "ت",
    "ﺗ": "ت",
    "ﺘ": "ت",
    "ﺙ": "ث",
    "ﺚ": "ث",
    "ﺛ": "ث",
    "ﺜ": "ث",
    "ﺝ": "ج",
    "ﺞ": "ج",
    "ﺟ": "ج",
    "ﺠ": "ج",
    "ﺡ": "ح",
    "ﺢ": "ح",
    "ﺣ": "ح",
    "ﺤ": "ح",
    "ﺥ": "خ",
    "ﺦ": "خ",
    "ﺧ": "خ",
    "ﺨ": "خ",
    "ﺩ": "د",
    "ﺪ": "د",
    "ﺫ": "ذ",
    "ﺬ": "ذ",
    "ﺭ": "ر",
    "ﺮ": "ر",
    "ﺯ": "ز",
    "ﺰ": "ز",
    "ﺱ": "س",
    "ﺲ": "س",
    "ﺳ": "س",
    "ﺴ": "س",
    "ﺵ": "ش",
    "ﺶ": "ش",
    "ﺷ": "ش",
    "ﺸ": "ش",
    "ﺹ": "ص",
    "ﺺ": "ص",
    "ﺻ": "ص",
    "ﺼ": "ص",
    "ﺽ": "ض",
    "ﺾ": "ض",
    "ﺿ": "ض",
    "ﻀ": "ض",
    "ﻁ": "ط",
    "ﻂ": "ط",
    "ﻃ": "ط",
    "ﻄ": "ط",
    "ﻅ": "ظ",
    "ﻆ": "ظ",
    "ﻇ": "ظ",
    "ﻈ": "ظ",
    "ﻉ": "ع",
    "ﻊ": "ع",
    "ﻋ": "ع",
    "ﻌ": "ع",
    "ﻍ": "غ",
    "ﻎ": "غ",
    "ﻏ": "غ",
    "ﻐ": "غ",
    "ﻑ": "ف",
    "ﻒ": "ف",
    "ﻓ": "ف",
    "ﻔ": "ف",
    "ﻕ": "ق",
    "ﻖ": "ق",
    "ﻗ": "ق",
    "ﻘ": "ق",
    "ﻙ": "ک",
    "ﻚ": "ک",
    "ﻛ": "ک",
    "ﻜ": "ک",
    "ﻝ": "ل",
    "ﻞ": "ل",
    "ﻟ": "ل",
    "ﻠ": "ل",
    "ﻡ": "م",
    "ﻢ": "م",
    "ﻣ": "م",
    "ﻤ": "م",
    "ﻥ": "ن",
    "ﻦ": "ن",
    "ﻧ": "ن",
    "ﻨ": "ن",
    "ﻩ": "ه",
    "ﻪ": "ه",
    "ﻫ": "ه",
    "ﻬ": "ه",
    "ﻭ": "و",
    "ﻮ": "و",
    "ﻯ": "ی",
    "ﻰ": "ی",
    "ﻱ": "ی",
    "ﻲ": "ی",
    "ﻳ": "ی",
    "ﻴ": "ی",
    "ﻵ": "لا",
    "ﻶ": "لا",
    "ﻷ": "لا",
    "ﻸ": "لا",
    "ﻹ": "لا",
    "ﻺ": "لا",
    "ﻻ": "لا",
    "ﻼ": "لا",
}

valid_chars = [
    " ",
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9",
    "A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U",
    "V",
    "W",
    "X",
    "Y",
    "Z",
    "a",
    "b",
    "c",
    "d",
    "e",
    "f",
    "g",
    "h",
    "i",
    "j",
    "k",
    "l",
    "m",
    "n",
    "o",
    "p",
    "q",
    "r",
    "s",
    "t",
    "u",
    "v",
    "w",
    "x",
    "y",
    "z",
    "آ",
    "ئ",
    "ا",
    "ب",
    "ت",
    "ث",
    "ج",
    "ح",
    "خ",
    "د",
    "ذ",
    "ر",
    "ز",
    "س",
    "ش",
    "ص",
    "ض",
    "ط",
    "ظ",
    "ع",
    "غ",
    "ف",
    "ق",
    "ل",
    "م",
    "ن",
    "ه",
    "و",
    "پ",
    "چ",
    "ژ",
    "ک",
    "گ",
    "ی",
]


def _replace_rep(t):
    "Replace repetitions at the character level: ccc -> c"

    def __replace_rep(m):
        c, cc = m.groups()
        return f"{c}"

    re_rep = re.compile(r"(\S)(\1{2,})")
    return re_rep.sub(__replace_rep, t)


def _replace_wrep(t):
    "Replace word repetitions: word word word -> word"

    def __replace_wrep(m):
        c, cc = m.groups()
        return f"{c}"

    re_wrep = re.compile(r"(\b\w+\W+)(\1{2,})")
    return re_wrep.sub(__replace_wrep, t)


def _normalize_text(x):
    """normalize a sentence"""

    x = str(x)
    x = parsivar_normalizer.normalize(x)  # apply `parsivar` normalizations
    x = re.sub(r"[\u200c\r\n]", " ", x)  # remove half space and new line characters
    x = x.lower()
    x = "".join(
        [char_mappings[xx] if xx in char_mappings else xx for xx in x]
    )  # substitue bad characters with appropriate ones
    x = re.sub(
        r"[^{}]".format("".join(valid_chars)), " ", x
    )  # just keep valid characters and substitue others with space
    x = re.sub(r"[a-z]+", r" \g<0> ", x)  # put space around words and numbers
    x = re.sub(r"[0-9]+", r" \g<0> ", x)  # put space around words and numbers
    x = re.sub(r"\s+", " ", x)  # remove more than one white spaces with space
    x = _replace_rep(x)
    x = _replace_wrep(x)
    return x.strip()


def normalize_texts(X, use_tqdm=False):
    """normalize list of sentences"""

    if use_tqdm:
        X = [_normalize_text(x) for x in tqdm(X)]
    else:
        X = [_normalize_text(x) for x in X]
    return X

## Prepare text features

In [ ]:
df_train.createOrReplaceTempView("DATA")

udf_text_procceing = udf(lambda x: ''.join(map(str, normalize_texts(x))) ,StringType())

In [ ]:
! pip install -q pyspark==3.1.2 spark-nlp
!pip install fasttext
!pip install hazm
import fasttext
import fasttext.util
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.ml import Pipeline

from sparknlp.base import *
from sparknlp.annotator import *

import parsivar

from __future__ import unicode_literals
import hazm

     |████████████████████████████████| 212.4 MB 73 kB/s 
     |████████████████████████████████| 198 kB 81.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
spark = sparknlp.start()

In [ ]:
normalizer_pars = parsivar.Normalizer()
normalizer_hazm = hazm.Normalizer()

In [ ]:
MODEL_NAME='dependency_typed_conllu'

document_assembler = DocumentAssembler() \
  .setInputCol("text") \
  .setOutputCol("document")

sentence_detector = SentenceDetector() \
  .setInputCols(["document"]) \
  .setOutputCol("sentence")

pos = PerceptronModel.pretrained("pos_ud_perdt", "fa") \
  .setInputCols(["document", "token"]) \
  .setOutputCol("pos")

pipeline = Pipeline(stages=[
  document_assembler,
  sentence_detector,
  posTagger
])

example = spark.createDataFrame([['سلام از John Ben Labs! ']], ["text"])

result = pipeline.fit(example).transform(example)



In [ ]:
desc = df_train.select("description").toPandas()

In [ ]:
for i in desc.description:
  i = normalizer_pars.normalize(i)
  i = normalizer_hazm.normalize(i)

In [ ]:
stop_words = pd.read_csv("/content/stopwords.csv", header=None)
stop_words.drop(columns=[1,2,3],inplace=True)

In [ ]:
for i in desc.description:
  for j in stop_words[0]:
    if j in i:
      i.replace(j,"")

In [ ]:
desc.head()

,description
0,بدون رنگ،کم کارکرد
1,اسفند 95\nبیمه بدنه کلا تخفیف \nمانیتور 10 این...
2,بیمه شخص ثالث،بیمه بدنه،دارای روکش صندلی و کفی...
3,■نمایندگی رجبی کد ۵٣٠٧٣٢\n□فروش نقدی ■ شعبه تو...
4,ماشین صفر برج ۹سال ۹۹ سند رهن ایران خودرو


In [ ]:
spark = sparknlp.start()
# fasttext.util.download_model('fa', if_exists='ignore')  # persian
# ft = fasttext.load_model('cc.fa.300.bin')

In [ ]:
embeddings = WordEmbeddingsModel.pretrained("persian_w2v_cc_300d", "fa") \  
        .setInputCols(["document", "token"]) \
        .setOutputCol("embeddings")
nlp_pipeline = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, embeddings])
pipeline_model = nlp_pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))
result = pipeline_model.transform(spark.createDataFrame([['من یادگیری ماشین را دوست دارم']], ["text"]))

In [ ]:
df_train.printSchema()

root
 |-- post_id: string (nullable = true)
 |-- review_label: integer (nullable = true)
 |-- body_status: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- brand_model: string (nullable = true)
 |-- category: string (nullable = true)
 |-- color: string (nullable = true)
 |-- description: string (nullable = true)
 |-- document: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- new_price: integer (nullable = true)
 |-- post_type: string (nullable = true)
 |-- selling_type: string (nullable = true)
 |-- third_party_insurance_deadline: integer (nullable = true)
 |-- tit le: string (nullable = true)
 |-- usage: integer (nullable = true)
 |-- year: string (nullable = true)



## Prepare categorical features

In [ ]:
df_train = df_train.fillna("0" ,['year'])
df_train = df_train.na.fill(0 ,['new_price'])
df_train = df_train.na.fill(-1 ,['usage'])

df_test = df_train.fillna("0" ,['year'])
df_test = df_test.na.fill(0 ,['new_price'])
df_test = df_test.na.fill(-1 ,['usage'])

In [ ]:
#change year val to number

df_train = df_train.withColumn("year", udf_text_procceing(col("year")))
df_train = df_train.withColumn("year",col("year").cast(IntegerType()))
df_train = df_train.fillna(0 ,['year'])

df_test = df_test.withColumn("year", udf_text_procceing(col("year")))
df_test = df_test.withColumn("year",col("year").cast(IntegerType()))
df_test = df_test.fillna(0 ,['year'])

In [ ]:
df_train = df_train.drop("third_party_insurance_deadline","category")    #all of this column is null so it's unusable    #category all is light
df_test = df_test.drop("third_party_insurance_deadline","category")    #all of this column is null so it's unusable    #category all is light

In [ ]:
## uncertain will change or use this columns

df_train = df_train.drop("brand_model","color","description","title","post_type")
df_test = df_test.drop("brand_model","color","description","title","post_type")

In [ ]:
target_cols = ['body_status',"brand","document","selling_type"]

target_numeric_cols =[]
for i in target_cols:
  target_numeric_cols.append(i+"_numeric")
  
df_train = df_train.fillna('0',target_cols)
df_test = df_test.fillna('0',target_cols)

indexer = StringIndexer(inputCols=target_cols ,outputCols=target_numeric_cols)

df_train = indexer.fit(df_train).transform(df_train)
df_test = indexer.fit(df_test).transform(df_test)

for column in target_cols:
  df_train = df_train.drop(column)
  df_test = df_test.drop(column)

for column in target_numeric_cols:
  df_train = df_train.withColumn(column,col(column).cast(IntegerType()))
  df_test = df_test.withColumn(column,col(column).cast(IntegerType()))

In [ ]:
features = df_train.drop("post_id")

## feature vector

In [ ]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors,VectorUDT

from catboost import CatBoostClassifier

In [ ]:
y = features.select("review_label").toPandas()
X = features.drop("review_label").toPandas()
cat_features = list(range(0,X.shape[1]))

##Train and Test model

In [ ]:
model = CatBoostClassifier(iterations=150)
model.fit(X,y,cat_features=cat_features,verbose=50) 

In [ ]:
model.score(X,y)

In [ ]:
x_predict = df_test.toPandas()
X_predict = x_predict.set_index("post_id")

In [ ]:
prediction = model.predict_proba(X_predict)

In [ ]:
result = []
for [T,F] in prediction:
  if T<=F :
    result.append(0)
  else:
    result.append(1)

In [ ]:
result = pd.DataFrame(result, columns = ['predictions'])
df_result = pd.concat([result, x_predict["post_id"]], axis=1, join="inner").set_index("post_id")
df_result.to_csv("/content/result.csv")
df_result.head()

In [ ]:
# spark.stop()

In [ ]:
# df_train.createOrReplaceTempView("DATA")
# spark.sql("SELECT third_party_insurance_deadline FROM Divar where third_party_insurance_deadline IS NOT NULL").show()

### First code about counting null values and second is about see which column that you want ###
# df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show() 
# df.select(['third_party_insurance_deadline','usage','review_label','description']).show()  
# df.na.fill('',['color']).show()

# spark.sql("SELECT * FROM DATA where year IS NULl").show()
# df.groupBy(df.brand).count().show()
# row = brand.distinct().count()
# df_train.summary().show()    //df.info()  in pandas

####################################
# farsi : 
#         post_type(optional)) ,   //
#         color(text processing) , //none
#         year(normaliaze number - 'utils') ,  //Done
#         title(text processing) , 
#         brand() //31 distinct values
# eng : 
#         brand_model  //none
# df.show()
# df.select(['post_id','review_label','body_status','brand_model','category','document','gearbox','new_price','selling_type','usage',"year"]).show() 